# Render and Analyze Predicted Genotypes

This notebook achieves the following:
1. Renders genotype prediction using Cluster Buster on GP2 non-GDPR release 7 data.
2. Processes imputed genotypes all the way from PLINK extraction to data transforming.
3. Processes WGS genotypes all the way from PLINK extraction to data transforming.
4. Compares predicted genotypes with imputed genotypes and WGS genotypes
5. Analyzes rates of concordance per snpID between predicted genotypes and imputed and WGS genotypes.

Imports

In [165]:
import os
import sys
import string 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pgenlib as pgenlib
import matplotlib.image as mpimg
import plotly.subplots as sp
import plotly.graph_objects as go
import string

# Functions

In [29]:
def filter_ndd_snps(ndd_genes, specific_genes, df):
    ndd_genes['chromosome'] = ndd_genes['chromosome'].str.replace('chr', '')
    ndd_genes['start_pos'] = pd.to_numeric(ndd_genes['start_pos'])
    ndd_genes['end_pos'] = pd.to_numeric(ndd_genes['end_pos'])

    filtered_gene_df = ndd_genes[ndd_genes['gene'].isin(specific_genes)].groupby('gene').first().reset_index()
    filtered_snps = pd.DataFrame(columns=df.columns)

    for _, gene_row in filtered_gene_df.iterrows():
        condition = (df['chromosome'] == gene_row['chromosome']) & \
                    (df['position'] >= gene_row['start_pos']) & \
                    (df['position'] <= gene_row['end_pos'])
        filtered_snps = pd.concat([filtered_snps, df[condition].assign(gene=gene_row["gene"])], ignore_index=True)

    filtered_snps.reset_index(drop=True, inplace=True)
    return filtered_snps

def calculate_A1counts(row, gt_column):
    # based on the following assumptions
    # a1 is always minor allele because frq file maf is never above 0.5
    # Ref from preds file always equals Ref from imputed files
    
    #if row["Ref"] == row["a1"]:
    if row["ALLELE_A"] == 0:
        if row[gt_column] == "AA":
            return 2.0
        elif row[gt_column] == "AB":
            return 1.0
        elif row[gt_column] == "BB":
            return 0.0
    #elif row["Ref"] == row["a2"]:
    elif row["ALLELE_A"] == 1:
        if row[gt_column] == "AA":
            return 0.0
        elif row[gt_column] == "AB":
            return 1.0
        elif row[gt_column] == "BB":
            return 2.0
    return None

def list_files_with_extension(directory, startswith, extension):
    files = os.listdir(directory)
    filtered_files = [file for file in files if file.startswith(startswith) and file.endswith(extension)]
    return filtered_files

def get_dfs_from_raw(raw_files_list, source_dir, suffix):
    dfs = pd.DataFrame()
    for file in raw_files_list:
        gt = pd.read_csv(f"{source_dir}/{file}", sep="\t")
        chr_columns = [col for col in gt.columns if col.startswith('chr') and not col.endswith('HET')]
        id_list = ["FID", "IID", "PAT", "MAT", "SEX", "PHENOTYPE"]
        melted = pd.melt(gt, id_vars=id_list, value_vars=chr_columns)        
        melted.rename(columns={"variable": f"snpID{suffix}", "value": f"A1counts{suffix}"}, inplace=True)
        dfs = pd.concat([dfs, melted], axis=0, ignore_index=True)

    return dfs

def get_chrom_position_ref_alt(dfs, snpid_column, suffix):
    dfs['chromosome'] = dfs[f'{snpid_column}'].str.split(':').str[0].str.replace('chr', '')
    dfs['position'] = dfs[f'{snpid_column}'].str.split(':').str[1]
    dfs[['chromosome', 'position']] = dfs[['chromosome', 'position']].astype('int64')
    
    dfs[f'Ref{suffix}'] = dfs[f'{snpid_column}'].str.split(':').str[2]
    dfs[f'Alt{suffix}'] = dfs[f'{snpid_column}'].str.split(':').str[3].str.split('_').str[0]
    return dfs

def complement_base(base):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return complement_dict.get(base, base)

def switch_A1(row, counts_column):
    if row[counts_column] == 2.0:
        return 0.0
    elif row[counts_column] == 0.0:
        return 2.0
    else:
        return 1.0

def merge_samples(predictions, new_genotypes, counts_column, new_genotypes_suffix):
    """
    From a dataframe containing GT_predicted and a dataframe containing GTs from another genotyping method (imputed or WGS),
    return a merged dataframe that considers switched reference and alternate alleles and ref and alt complements 
    Parameters:
    predictions - dataframe containing predicted genotypes, usually output of predictions.py
    new_genotypes - dataframe containing imputed or WGS genotypes
    new_genotypes_suffix - usually "_imputed" or "_wgs", use same suffix as suffix parameter in get_dfs_from_raw()
    Returns:
    pandas dataframe with inner merge of predicted and new genotypes
    """
    first = predictions.merge(new_genotypes, on=["Sample_ID", "chromosome", "position"], how="left")

    # Matched refs and alts
    matched_refs = first.loc[(first["Ref"] == first[f"Ref{new_genotypes_suffix}"]) & (first["Alt"] == first[f"Alt{new_genotypes_suffix}"])].copy()
    matched_refs["merge_id"] = 1
    
    # Different refs and alts
    different_refs = first.loc[(first["Ref"] != first[f"Ref{new_genotypes_suffix}"]) | (first["Alt"] != first[f"Alt{new_genotypes_suffix}"])].copy()
    
    # Match Ref to Alt_imputed and Alt to Ref_imputed, then switch A1counts_imputed
    matched_ref_to_alt = different_refs.loc[(different_refs["Ref"] == different_refs[f"Alt{new_genotypes_suffix}"]) & (different_refs["Alt"] == different_refs[f"Ref{new_genotypes_suffix}"])].copy()
    matched_ref_to_alt[f"A1counts{new_genotypes_suffix}"] = matched_ref_to_alt.apply(switch_A1, counts_column=counts_column, axis=1)
    matched_ref_to_alt["merge_id"] = 2
    
    # Take care of complement strand possibilities
    different_refs[f"Ref{new_genotypes_suffix}_complement"] = different_refs[f"Ref{new_genotypes_suffix}"].apply(complement_base)
    different_refs[f"Alt{new_genotypes_suffix}_complement"] = different_refs[f"Alt{new_genotypes_suffix}"].apply(complement_base)
    
    matched_refs_complement = different_refs.loc[(different_refs["Ref"] == different_refs[f"Ref{new_genotypes_suffix}_complement"]) & (different_refs["Alt"] == different_refs[f"Alt{new_genotypes_suffix}_complement"])].copy()
    matched_refs_complement["merge_id"] = 3
    
    matched_ref_to_alt_complement = different_refs.loc[(different_refs["Ref"] == different_refs[f"Alt{new_genotypes_suffix}_complement"]) & (different_refs["Alt"] == different_refs[f"Ref{new_genotypes_suffix}_complement"])].copy()
    matched_ref_to_alt_complement[f"A1counts{new_genotypes_suffix}"] = matched_ref_to_alt_complement.apply(switch_A1, counts_column=counts_column, axis=1)
    matched_ref_to_alt_complement["merge_id"] = 4
    
    compare = pd.concat([matched_refs, matched_ref_to_alt, matched_refs_complement, matched_ref_to_alt_complement], axis=0)
    
    compare.drop_duplicates(subset=["Sample_ID", "snpID", "chromosome", "position"], inplace=True)
    return compare

def replace_colons(snp_id):
    parts = snp_id.rsplit('_', 1)  # Split into two parts from the right
    parts[0] = parts[0].replace('_', ':')  # Replace ':' with '_' in the first part
    return '_'.join(parts)

def determine_concordant_summary(row):
    if row['concordant_wgs'] == True:
        return True
    if row['concordant_wgs'] == False:
        return False
    if pd.isnull(row["concordant_wgs"]) and row["concordant_imputed"] == True:
        return True
    if pd.isnull(row["concordant_wgs"]) and row["concordant_imputed"] == False:
        return False
    if pd.isnull(row["concordant_wgs"]) and pd.isnull(row["concordant_imputed"]):
        return np.NaN

def calculate_GT(row, counts_column):
    # Reverse the calculation based on A1counts_predicted
    if row["ALLELE_A"] == 0:
        if row[counts_column] == 2.0:
            return "AA"
        elif row[counts_column] == 1.0:
            return "AB"
        elif row[counts_column] == 0.0:
            return "BB"
    elif row["ALLELE_A"] == 1:
        if row[counts_column] == 0.0:
            return "AA"
        elif row[counts_column] == 1.0:
            return "AB"
        elif row[counts_column] == 2.0:
            return "BB"
    return None

def convert_GT_to_cat(data, gt_col_name, new_col_name):
    # apply gt_cat() mapping function to add new column with 0/1/2/NaN instead of AA/AB/BB/NC
    # data[f"{gt_col_name}_{new_col_suffix}"] = data.apply(lambda row: gt_cat(row, gt_col_name), axis=1)
    data[f"{new_col_name}"] = data[f"{gt_col_name}"].map({"AA":0, "AB":1, "BA":1, "BB":2, "NC":np.NaN})
    return data

# Combine Cluster Buster Predicted Genotypes with Imputed and WGS Genotypes

## Extract list of SNPs within APOE, GBA, SNCA, LRRK2 regions.

ndd_file contains list of neurodegenerative disease related genes along with their chromosome and position.

In [ ]:
ndd_file = "data/ndd_genes_pos_uniq.txt"
cols = ["gene", "chromosome", "start_pos", "end_pos"]
ndd_genes = pd.read_csv(ndd_file, sep=",", names=cols)

In [ ]:
specific_genes = ['SNCA', 'APOE', 'GBA', 'LRRK2']
                    
example = "/data/GP2/raw_genotypes/snp_metrics/204842400049/snp_metrics_204842400049/Sample_ID=204842400049_R01C01"
snps = pd.read_parquet(example)
snps = snps[["snpID","Ref","Alt","chromosome","position"]]

Get snpIDs within specific genes by filtering a on chromosome and position.

In [ ]:
filtered_snps = filter_ndd_snps(ndd_genes, specific_genes, snps)

Save list of SNPs to extract to a txt file

In [ ]:
filtered_snps["snpID"].to_csv('data/fresh_run/extract_snpids.txt', index=False, header=False)

## Get Predictions on Release 7

Aggregate SNPs within Specific Gene Loci

In [ ]:
data_dir = "/data/GP2/raw_genotypes/snp_metrics"
snpids = "/data/CARD_AA/projects/2023_05_JM_gt_clusters/capstone/data/extract_snpids.txt"
output_dir = "/data/CARD_AA/projects/2023_05_JM_gt_clusters/capstone/data/release7_sampleids"

with open(f'release7_sampleids/agg_snps.swarm', 'w') as f:
    for item in os.listdir('release7_sampleids'):
        if item.startswith('release7'):
            sample = f"{output_dir}/{item}"
            output_suffix = item.split('_')[2]
            output_file = f"{output_dir}/agg_snps_{output_suffix}.csv"
            cmd = f"\
python /data/CARD_AA/projects/2023_05_JM_gt_clusters/capstone/src/aggregate_snps.py \
--ids {snpids} \
--samples {sample} \
--directory {data_dir} \
--output {output_file}"
            f.write(f'{cmd}\n')
f.close()

Run code to aggregate SNPs

In [ ]:
! swarm -g 5 -t 8 --time=03:00:00 release7_sampleids/agg_snps.swarm

Render predictions

In [ ]:
! bash src/predictions.sh model/gt_model.keras data/snpid_map.txt data/release7_snps.csv data/release7_predictions.csv

Load predictions.

In [13]:
fpath = "data/release7_predictions.csv"
predictions = pd.read_csv(fpath, sep=",")

Calculate counts of a1 allele (A1counts_predicted) from GT_predicted.

In [14]:
predictions["A1counts_predicted"] = predictions.apply(calculate_A1counts_predicted, gt_column="GT_predicted", axis=1)

## Process Imputed Genotypes

### Establish samples of interest

Merge relevant information from master keys for release 7 information.

In [ ]:
key_file = f'/data/GP2/clinical/master_key/release_keys/master_key_release7_final.csv'
master_key = f"/data/GP2/clinical/master_key/GP2_master_key_JUNE_2024.txt"
key = pd.read_csv(f'{key_file}', sep=",")
master = pd.read_csv(f'{master_key}', sep="\t")

master = master[["GP2sampleID", "SentrixBarcode_A", "SentrixPosition_A"]].astype(str)
key = key.merge(master, left_on="GP2sampleID", right_on="GP2sampleID", how="left")

In [ ]:
key['filename'] = key['SentrixBarcode_A'].astype(str) + '_' + key['SentrixPosition_A'].astype(str)

Save samples of interest to text file.

In [ ]:
key['filename'] = key['SentrixBarcode_A'].astype(str) + '_' + key['SentrixPosition_A'].astype(str)
sampleids_list = key['filename'].tolist()
with open('data/release7_sampleids.txt', 'w') as file:
    for sampleid in sampleids_list:
        file.write(sampleid + '\n')
file.close()

### Extract Imputation with PLINK

In [ ]:
! bash extract_imputed_gts.sh

### Transform Imputed Genotypes into Long Form

Transforms wide form imputed data into long form for ease in merging with predicted data later.

This is often a very computationally expensive process and must be run one gene at a time for memory/RAM preservation.

Set the gene variable equal to one of the genes in gene_regions list to process imputed data for one gene.

In [26]:
gene_regions = ["GBA", "SNCA", "LRRK2", "APOE"]
gene_dict = {"GBA":1, "SNCA":4, "LRRK2":12, "APOE":19}

gene = "APOE"

In [27]:
directory = 'data/new_new_imputed_genotypes'
chrom = f"chr{gene_dict[gene]}_"
extension = ".raw"
raw_files_imputed = list_files_with_extension(directory, chrom, extension)

Get imputed data from each raw file into one dataframe (dfs_imputed)

In [28]:
dfs_imputed = get_dfs_from_raw(raw_files_imputed, "data/new_new_imputed_genotypes", "_imputed")

In [29]:
dfs_imputed = get_chrom_position_ref_alt(dfs_imputed, "snpID_imputed", "_imputed")

In [30]:
dfs_imputed['chromosome'] = dfs_imputed['snpID_imputed'].str.split(':').str[0].str.replace('chr', '')
dfs_imputed['position'] = dfs_imputed['snpID_imputed'].str.split(':').str[1]
dfs_imputed[['chromosome', 'position']] = dfs_imputed[['chromosome', 'position']].astype('int64')

dfs_imputed['Ref_imputed'] = dfs_imputed['snpID_imputed'].str.split(':').str[2]
dfs_imputed['Alt_imputed'] = dfs_imputed['snpID_imputed'].str.split(':').str[3].str.split('_').str[0]

Add GP2 IIDs to dfs for merging with predicted data later.

In [31]:
fpath = "/data/GP2/clinical/master_key/GP2_master_key_JUNE_2024.txt"
master_key = pd.read_csv(fpath, sep="\t")
master_key.rename(columns={"IID":"Sample_ID"}, inplace=True)
ids = master_key[["GP2sampleID", "Sample_ID"]]
dfs_imputed = dfs_imputed.merge(ids, left_on="IID", right_on="GP2sampleID", how="left")

/tmp/ipykernel_1423337/3785646006.py:2: DtypeWarning: Columns (23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  master_key = pd.read_csv(fpath, sep="\t")


Round A1counts_imputed column for comparing with other genotypes.

In [32]:
dfs_imputed["A1counts_imputed_rounded"] = dfs_imputed["A1counts_imputed"].round()

### Merge Imputed Genotypes with Predicted Genotypes

Merge data based on matching Sample_IDs, chromosome, position, reference allele, and alternate allele. Multiple merges are required (flipping reference and alternate, considering complement alleles) in order to ensure all possible matches have been made. Sometimes the panels measure different alleles and different strands.

In [33]:
compare_imputed = merge_samples(predictions, dfs_imputed, "A1counts_imputed", "_imputed")

In [34]:
compare_imputed.to_csv(f"data/fresh_run/{gene}_predicted_vs_imputed_uncleaned.csv", index=False)

Clean out SNPs with poor raw data.

In [35]:
compare_imputed = compare_imputed.dropna(subset=["R", "Theta"], axis=0)
compare_imputed = compare_imputed.loc[compare_imputed["R"] > 0.15].copy()

Save merged predictions and imputations for the gene.

In [36]:
compare_imputed.to_csv(f"data/fresh_run/{gene}_predicted_vs_imputed.csv", index=False)

### Concatenate predicted_vs_imputed files for each gene and save to csv

In [42]:
imputed = pd.DataFrame()
for gene in gene_regions:
    gene_df = pd.read_csv(f"data/fresh_run/{gene}_predicted_vs_imputed.csv", sep=",")
    imputed = pd.concat([imputed, gene_df], axis=0, ignore_index=True)

imputed.to_csv("data/fresh_run/release7_predicted_vs_imputed.csv", index=False)

## Process WGS Genotypes

### Extract WGS data with PLINK

In [ ]:
wgs_dir = "/data/CARD_AA/projects/2023_05_JM_gt_clusters/wgs_snps"
names = ["wgs_apoe_snps", "wgs_gba_snps", "wgs_snca_snps", "wgs_lrrk2_snps"]
for name in names:
    ! echo {name}
    ! plink2 --pfile {wgs_dir}/{name} --recode AD --out data/wgs/{name}

### Transform Imputed Genotypes into Long Form

In [6]:
directory = '/data/CARD_AA/projects/2023_05_JM_gt_clusters/capstone/data/wgs'
extension = ".raw"
startswith = "wgs"
raw_files_wgs = list_files_with_extension(directory, startswith, extension)

dfs_wgs = get_dfs_from_raw(raw_files_wgs, "data/wgs", "_wgs")

Harmonize snpIDs to match imputed snpID format (chrN:pos:Ref:Alt_Ref) 

In [9]:
dfs_wgs['snpID_wgs'] = dfs_wgs['snpID_wgs'].apply(replace_colons)

Get sample information into dataframe

In [10]:
release = pd.read_csv("/data/GP2/clinical/master_key/GP2_master_key_JULY_2024.txt", sep='\t')
release.rename(columns={"IID":"Sample_ID"}, inplace=True)
ids = release[["GP2sampleID", "Sample_ID"]]
dfs_wgs = dfs_wgs.merge(ids, left_on="IID", right_on="GP2sampleID", how="left")

/tmp/ipykernel_2625756/3298413311.py:1: DtypeWarning: Columns (23,24,25,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  release = pd.read_csv("/data/GP2/clinical/master_key/GP2_master_key_JULY_2024.txt", sep='\t')


Transform data to extract chromosome, position, reference, alternate allele.

### Merge WGS Genotypes with Predicted Genotypes

In [21]:
compare_wgs = merge_samples(predictions, dfs_wgs, "A1counts_wgs", "_wgs")

In [22]:
compare_wgs.to_csv("data/fresh_run/release7_predicted_vs_wgs.csv", index=False)

## Merge of Predicted, Imputed, and WGS Genotypes

Load imputed and WGS dataframe if necessary.

In [26]:
compare_imputed = pd.read_csv("data/fresh_run/release7_predicted_vs_imputed.csv", sep=",")
compare_wgs = pd.read_csv("data/fresh_run/release7_predicted_vs_wgs.csv", sep=",")

From A1counts, make GT (GT_wgs, GT_imputed) column and categorical GT column (GT_wgs_cat, GT_imputed_cat)

In [30]:
compare_imputed["GT_imputed"] = compare_imputed.apply(calculate_GT, counts_column="A1counts_imputed", axis=1)
compare_wgs["GT_wgs"] = compare_wgs.apply(calculate_GT, counts_column="A1counts_wgs", axis=1)

In [33]:
compare_imputed = convert_GT_to_cat(compare_imputed, gt_col_name="GT_imputed", new_col_name="GT_imputed_cat")
compare_wgs = convert_GT_to_cat(compare_wgs, gt_col_name="GT_wgs", new_col_name="GT_wgs_cat")

Reduce the dataframe holding WGS information (compare_wgs) and the dataframe holding imputed information (compare_imputed) to most important columns.

In [36]:
wgs_important_columns = ['snpID', 'Sample_ID', 'snpID_wgs', 'A1counts_wgs', 'GT_wgs', 'GT_wgs_cat']
wgs_important = compare_wgs[wgs_important_columns]

imputed_important_columns = ['snpID', 'Sample_ID', 'snpID_imputed', 'A1counts_imputed', 'A1counts_imputed_rounded', 'GT_imputed', 'GT_imputed_cat']
imputed_important = compare_imputed[imputed_important_columns]

Perform two left merges to get a dataframe (compare_gts) that contains all samples that have either imputed genotypes, WGS genotypes, or both.

In [55]:
# left merge
preds_intermed = predictions.merge(wgs_important, on=["snpID", "Sample_ID"], how="left")
df = preds_intermed.merge(imputed_important, on=["snpID", "Sample_ID"], how="left")

Drop rows with no imputation or WGS.

In [56]:
df = df.dropna(subset=["A1counts_wgs", "A1counts_imputed"], how='all')

Add a gene column.

In [83]:
gene_map = {1: "GBA", 4:"SNCA", 12:"LRRK2", 19:"APOE"}
df["gene"] = df["chromosome"].map(gene_map)

### Create Overall Concordance Metric

Create True/False columns that indicate if the predicted GT is in agreement (concordant) with the imputed GT and the WGS GT. For SNPs with a NaN, change False to NaN.

In [59]:
df['concordant_wgs'] = df["A1counts_predicted"] == df["A1counts_wgs"]
df['concordant_imputed'] = df["A1counts_predicted"] == df["A1counts_imputed_rounded"]

df['concordant_wgs'] = df['concordant_wgs'].mask(df["A1counts_wgs"].isna(), np.nan)
df['concordant_imputed'] = df['concordant_imputed'].mask(df["A1counts_imputed"].isna(), np.nan)

Create a True/False column concordance_summary that prioritizes concordance with WGS GTs over imputed GTs.

In [60]:
df['concordant_summary'] = df.apply(determine_concordant_summary, axis=1)

Save to CSV

In [84]:
df.to_csv("data/fresh_run/release7_predictions_imputations_wgs.csv", index=False)

Separate out previous no-calls and save to CSV.

In [127]:
df_nc = df.loc[df["GT"] == "NC"]
df_nc.to_csv("data/fresh_run/release7_nocalls_predictions_imputations_wgs.csv", index=False)

# Analysis

If necessary, load dataframe with predictions, imputation, and WGS.

In [ ]:
df_nc = pd.read_csv("data/fresh_run/release7_nocalls_predictions_imputations_wgs.csv", sep=",")

## Summarize SNP Concordance

Create a summary dataframe with counts and mean rates of concordance per SNP.

In [128]:
imputed_concordance_df = df_nc.groupby('snpID')['concordant_imputed'].mean() * 100
imputed_concordance_df = imputed_concordance_df.to_frame().reset_index()

In [129]:
wgs_concordance_df = df_nc.groupby('snpID')['concordant_wgs'].mean() * 100
wgs_concordance_df = wgs_concordance_df.to_frame().reset_index()

In [130]:
summary_concordance_df = df_nc.groupby('snpID')['concordant_summary'].mean() * 100
summary_concordance_df = summary_concordance_df.to_frame().reset_index()

In [131]:
summary_df = imputed_concordance_df.merge(wgs_concordance_df, on="snpID", how="outer")
summary_df = summary_df.merge(summary_concordance_df, on="snpID", how="outer")

In [132]:
imputed_counts_df = df_nc.groupby('snpID')['snpID_imputed'].apply(lambda x: x.notna().sum()).reset_index(name='counts_imputed')
summary_df = summary_df.merge(imputed_counts_df, on="snpID", how="outer")

In [133]:
wgs_counts_df = df_nc.groupby('snpID')['snpID_wgs'].apply(lambda x: x.notna().sum()).reset_index(name='counts_wgs')
summary_df = summary_df.merge(wgs_counts_df, on="snpID", how="outer")

Add gene to summary_df to help with later analysis.

In [134]:
extra_info = df_nc[["snpID","gene"]]
extra_info = extra_info.drop_duplicates()
summary_df = summary_df.merge(extra_info, on="snpID", how="inner")

In [135]:
summary_df.to_csv("data/fresh_run/summary_concordance.csv", index=False)

## Define High-Performing SNPS

High-performing SNPs are SNPs for which Cluster Buster predicted genotypes are in concordance with imputed GTs and/or WGS at a rate of at least 90%.

In [137]:
threshold = 90

high_performing = summary_df.loc[
    ((summary_df["concordant_wgs"] >= threshold) | pd.isna(summary_df["concordant_wgs"])) &
    ((summary_df["concordant_imputed"] >= threshold) | pd.isna(summary_df["concordant_imputed"])) &
    ((summary_df["concordant_imputed"] >= threshold) | (summary_df["concordant_wgs"] >= threshold))
]

## Calculate New Call Rate for High-Performing SNPs

Load missingness for release 7 and merge with high performing SNPs.

In [138]:
miss = pd.read_csv("data/jessica_release7_miss.lmiss", sep="\s+")
miss["original_call_rate"] = 1 - miss["F_MISS"]

In [139]:
high_performing = high_performing.merge(miss, left_on="snpID", right_on="SNP", how="left")

Narrow down predictions to previous no-calls and predicted genotypes are concordant with imputation/WGS and add counts to high performing snp dataframe.

In [140]:
trusted_predictions = df_nc.loc[df_nc["snpID"].isin(high_performing["snpID"].unique())]
trusted_predictions.columns
correct_predictions = trusted_predictions[trusted_predictions['concordant_summary'] == True]
counts = correct_predictions.groupby("snpID").size().astype(int)
counts = counts.to_frame(name="new_counts").reset_index()

high_performing = high_performing.merge(counts, on="snpID", how="left")

Calculate new call rate (new_call_rate) for release 7 when well predicted genotypes from Cluster Buster are added.

In [141]:
high_performing["N_MISS_NEW"] = high_performing["N_MISS"] - high_performing["new_counts"]
high_performing["F_MISS_NEW"] = high_performing["N_MISS_NEW"] / high_performing["N_GENO"]
high_performing["new_call_rate"] = 1 - high_performing["F_MISS_NEW"]

Calculate difference in call rate between old and new call rate.

In [142]:
high_performing["call_rate_difference"] = high_performing["new_call_rate"] - high_performing["original_call_rate"]
high_performing = high_performing.sort_values(by="call_rate_difference", ascending=False)

In [147]:
high_performing["gene"].value_counts()

gene
LRRK2    43
APOE     30
GBA      28
Name: count, dtype: int64

In [153]:
high_performing.to_csv("data/fresh_run/high_performing_snps.csv", index=False)

## Explore SNPs Recovered at Various Concordance Thresholds

In [144]:
thresholds = [50, 60, 70, 80, 85, 90, 95]

results = pd.DataFrame()

results = summary_df['gene'].value_counts().reset_index()
results.columns = ['gene', 'original_count']

for threshold in thresholds:
    filtered_df = summary_df.loc[
        ((summary_df["concordant_wgs"] >= threshold) | pd.isna(summary_df["concordant_wgs"])) &
        ((summary_df["concordant_imputed"] >= threshold) | pd.isna(summary_df["concordant_imputed"])) &
        ((summary_df["concordant_imputed"] >= threshold) | (summary_df["concordant_wgs"] >= threshold))
    
    ]
    
    counts = filtered_df["gene"].value_counts().reset_index()
    counts.columns = ['gene', f'count_threshold_{threshold}']
    
    if results.empty:
        results = counts
    else:
        results = pd.merge(results, counts, on='gene', how='outer')

results = results.fillna(0)
for col in results.columns:
    if col.startswith("count_threshold_"):
        results[col] = results[col].astype(int)

In [152]:
results.to_csv("data/fresh_run/snp_counts_recovered_thresholds.csv", index=False)